In [1]:
import pandas as pd
from datetime import datetime

In [2]:
# Load the data
df = pd.read_csv("supermarket.csv")
customer_df = pd.read_csv("customers.csv")

# Convert the order date to datetime
df["Order Date"] = pd.to_datetime(df["Order Date"])
df["Ship Date"] = pd.to_datetime(df["Ship Date"])

In [3]:
# Define the snapshot date as the max Order Date + 1 day
snapshot_date = df["Order Date"].max() + pd.Timedelta(days=1)

# Recency is calculated as the number of days between the customer's latest order and the most recent order in the dataset.
# Frequency is calculated as the number of orders made by the customer.
# Monetary Value is calculated as the total sales from the customer.

# Calculate Recency, Frequency and Monetary value for each customer
customer_profile = (
    df.groupby("Customer ID")
    .agg(
        {
            "Order Date": lambda x: (snapshot_date - x.max()).days,  # Recency
            "Order ID": "count",  # Frequency
            "Sales": "sum",  # Monetary Value
        }
    )
    .reset_index()
)

# Rename the columns
customer_profile.rename(
    columns={
        "Order Date": "Recency",
        "Order ID": "Frequency",
        "Sales": "MonetaryValue",
    },
    inplace=True,
)

# Create labels for Recency, Frequency and MonetaryValue
r_labels = range(4, 0, -1)
f_labels = range(1, 5)
m_labels = range(1, 5)

# Assign these labels to four equal percentile groups
r_groups = pd.qcut(customer_profile["Recency"], q=4, labels=r_labels)
f_groups = pd.qcut(customer_profile["Frequency"], q=4, labels=f_labels)
m_groups = pd.qcut(customer_profile["MonetaryValue"], q=4, labels=m_labels)

# Create new columns R, F, M in the dataframe
customer_profile = customer_profile.assign(
    R=r_groups.values, F=f_groups.values, M=m_groups.values
)

# Concatenate RFM quartile values to RFM_Segment
customer_profile["RFM_Segment"] = customer_profile.apply(
    lambda x: str(x["R"]) + str(x["F"]) + str(x["M"]), axis=1
)

# Sum RFM quartiles values to RFM_Score
customer_profile["RFM_Score"] = customer_profile[["R", "F", "M"]].sum(axis=1)


# Define rfm_level function
def rfm_level(df):
    if df["RFM_Score"] >= 10:
        return "High Value Customer"
    elif (df["RFM_Score"] < 10) & (df["RFM_Score"] >= 6):
        return "Mid Value Customer"
    else:
        return "Low Value Customer"


# Create a new variable RFM_Level
customer_profile["RFM_Level"] = customer_profile.apply(rfm_level, axis=1)

In [4]:
# Assuming 'birthday' is in the format 'YYYY-MM-DD'
customer_df["Birthday"] = pd.to_datetime(customer_df["Birthday"])

# Calculate the current year
current_year = datetime.now().year

# Extract the year from 'birthday' column
customer_df["year"] = customer_df["Birthday"].dt.year

# Calculate age
customer_df["Age"] = current_year - customer_df["year"]

# Merge 'customer_df' and 'customer_profile' on 'customer_code' and 'customer_id'
customer_profile = customer_profile.merge(
    customer_df[["Customer Code ", "Age"]],
    left_on="Customer ID",
    right_on="Customer Code ",
    how="left",
)
# After the merge, drop the 'customer_code' column
customer_profile = customer_profile.drop("Customer Code ", axis=1)

In [5]:
# Calculate the Contribution Margin per order
df["Contribution Margin"] = df["Revenue"] - (
    df["COGS"] + df["Expense"] + df["Discount"] + df["Shipping cost"]
)

# Calculate the Contribution Margin per customer
clv_grouped = (
    df.groupby("Customer ID").agg({"Contribution Margin": "sum"}).reset_index()
)

customer_profile = pd.merge(customer_profile, clv_grouped, on="Customer ID", how="left")

In [6]:
# Calculate the maximum order date (latest date in the dataset)
max_order_date = df["Order Date"].max()

# Calculate the maximum order date per customer
customer_churn = df.groupby("Customer ID")["Order Date"].max().reset_index()
customer_churn.rename(columns={"Order Date": "Latest Order Date"}, inplace=True)

# Define churn period (e.g., 6 months)
churn_period = pd.Timedelta("180 days")

# Store the max_order_date in the DataFrame
customer_churn["Max Order Date"] = max_order_date

# Calculate the difference between max_order_date and customer_churn['Order Date']
customer_churn["Order Difference"] = (
    customer_churn["Max Order Date"] - customer_churn["Latest Order Date"]
)

# Create an "Is Churn" column in customer_churn DataFrame
customer_churn["Is Churn"] = customer_churn["Order Difference"] >= churn_period

# Drop 'Order Difference' and 'Max Order Date' columns
customer_churn.drop(
    columns=["Order Difference", "Max Order Date", "Latest Order Date"], inplace=True
)

# Convert boolean values to 1 (churn) or 0 (not churn)
customer_churn["Is Churn"] = customer_churn["Is Churn"].astype(int)

customer_profile = pd.merge(
    customer_profile, customer_churn, on="Customer ID", how="left"
)

In [7]:
# Calculate Duration of customer relationship
duration = df.groupby("Customer ID")["Order Date"].agg(["min", "max"]).reset_index()
duration["Duration"] = (duration["max"] - duration["min"]).dt.days
duration.drop(columns=["min", "max"], inplace=True)

customer_profile = pd.merge(customer_profile, duration, on="Customer ID", how="left")

In [8]:
# Calculate total revenue per customer
revenue = df.groupby("Customer ID")["Sales"].sum().reset_index(name="Total Revenue")

# Merge this dataframe with customer_profile
customer_profile = pd.merge(customer_profile, revenue, on="Customer ID", how="left")

In [9]:
# Calculate total quantity per customer
quantity = (
    df.groupby("Customer ID")["Quantity"].sum().reset_index(name="Total Quantity")
)

# Merge this dataframe with customer_profile
customer_profile = pd.merge(customer_profile, quantity, on="Customer ID", how="left")

In [10]:
# Calculate total profit per customer
profit = df.groupby("Customer ID")["Profit"].sum().reset_index(name="Total Profit")

# Merge this dataframe with customer_profile
customer_profile = pd.merge(customer_profile, profit, on="Customer ID", how="left")

In [11]:
# Calculate total discount per customer
discount = (
    df.groupby("Customer ID")["Discount N"].sum().reset_index(name="Total Discount")
)

# Merge this dataframe with customer_profile
customer_profile = pd.merge(customer_profile, discount, on="Customer ID", how="left")

In [12]:
# Calculate total shipping cost per customer
shipping_cost = (
    df.groupby("Customer ID")["Shipping cost"]
    .sum()
    .reset_index(name="Total Shipping Cost")
)

# Merge this dataframe with customer_profile
customer_profile = pd.merge(
    customer_profile, shipping_cost, on="Customer ID", how="left"
)

In [13]:
# Calculate shipping duration per order
df["Shipping Duration"] = (df["Ship Date"] - df["Order Date"]).dt.days

# Calculate average shipping duration per customer
avg_shipping_duration = (
    df.groupby("Customer ID")["Shipping Duration"]
    .mean()
    .reset_index(name="Average Shipping Duration")
)

# Merge this dataframe with customer_profile
customer_profile = pd.merge(
    customer_profile, avg_shipping_duration, on="Customer ID", how="left"
)

In [14]:
# List of columns to count unique values
columns_to_count_unique = [
    "Category",
    "Sub-Category",
    "Vendor Name",
    "Product ID",
    "Segment",
    "Customer Type",
    "Member card",
    "Region",
    "Province ",
    "Channel",
    "Store ID",
    "Ship Mode",
    "Shipper",
]

# Loop over the columns and count unique values for each customer
for column in columns_to_count_unique:
    unique_counts = (
        df.groupby("Customer ID")[column].nunique().reset_index(name="count")
    )

    # If any customer has a count > 1 for the column, store the count for the entire column
    if unique_counts["count"].max() > 1:
        unique_counts[f"{column} Unique Counts"] = unique_counts["count"]
    # If all customers have a count of 1, store the value
    else:
        last_order_value = (
            df.sort_values("Order Date", ascending=False)
            .groupby("Customer ID")[column]
            .first()
            .reset_index()
        )
        # Merge last_order_value with unique_counts
        unique_counts = pd.merge(unique_counts, last_order_value, on="Customer ID")

    # Drop the 'count' column
    unique_counts.drop(columns=["count"], inplace=True)

    # Merge this dataframe with customer_profile
    customer_profile = pd.merge(
        customer_profile, unique_counts, on="Customer ID", how="left"
    )

In [15]:
customer_profile.to_csv("customer_profile.csv", index=False)